In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [11]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [12]:
idx = 0

In [13]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [ ]:
# chromedriver_path = 'C:/Users/kkksk/.wdm/drivers/chromedriver/win64/122.0.6261.129/chromedriver.exe'
# service = Service(chromedriver_path)
# driver = webdriver.Chrome(service=service)

In [7]:
# cur_month = 1
cur_year = 2002
target_year = 2005

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=brent+oil&tbs=bkt:s,cdr:1,cd_min:1/1/{cur_year},cd_max:12/31/{cur_year}&tbm=bks"
    driver.get(url)
    
    time.sleep(30)
    
    for _ in range(16):
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso>div')
        if not news_results: 
            break  
            
        for news_div in news_results:
            try:
                Link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                title_element = news_div.find_element(By.CSS_SELECTOR, 'a>h3')
                Title = title_element.text.strip(".")
                Title = re.sub(r"- \d+페이지", "", Title)
                news_elements = news_div.find_elements(By.CSS_SELECTOR, 'div')
                Date = ''  # Default Date value if not found
                if len(news_elements) >= 10:
                    info = news_elements[10].text  # Assuming 'info' is in the 11th element
                    info_text = info.split("·")
                    if len(info_text) > 1:  # Check if Date is available
                        Date = info_text[1].strip()
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1

            except Exception as e:
                print(e)
            
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(1)
        except NoSuchElementException:
            break
    cur_year+=1
    # if cur_month == 7:  
    #     cur_month = 1
    #     cur_year += 1
    # else: 
    #     cur_month += 6

In [8]:
new_df

,title,url,date
0,Possibility Of War With Iraq Could Make Oil St...,https://books.google.co.kr/books?id=ZZ5dAAAAIB...,2002. 12. 8.
1,Expert Says Gas Prices Should Decline Soon,https://books.google.co.kr/books?id=FHo0AAAAIB...,2002. 4. 10.
2,Commodity Rdp. Crude Futures Jump To Two-Year ...,https://books.google.co.kr/books?id=2W0iAAAAIB...,2002. 12. 27.
3,Rising Oil Prices Begin To Affect Us Consumers.,https://books.google.co.kr/books?id=7G8rAAAAIB...,2002. 12. 30.
4,Oil Prices Stable Though High Due To Mideast W...,https://books.google.co.kr/books?id=znRhAAAAIB...,2002. 4. 4.
...,...,...,...
635,Storm Seen As Catalyst For Already High Energy...,https://books.google.co.kr/books?id=CAJaAAAAIB...,2005. 8. 29.
636,"Demand, Threats Pump Oil Prices To Record. By ...",https://books.google.co.kr/books?id=eBFPAAAAIB...,2005. 8. 8.
637,The Free Lance-Star,https://books.google.co.kr/books?id=2TMzAAAAIB...,2005. 8. 20.
638,New York Crude Oil Ends Down On Supply Signals.,https://books.google.co.kr/books?id=mP1GAAAAIB...,2005. 10. 4.


In [10]:
new_df.to_parquet('02~23/crude_books_2002_2005.parquet', engine = 'pyarrow', compression = 'gzip', index = False)